## LHIPA calculation

### input file:

### output file:

In [2]:
import pandas as pd
import numpy as np
import math, pywt

In [39]:
def modmax(d): 
    # compute signal modulus 
    m = [0.0]*len(d) 
    for i in range(len(d)): 
        m[i] = math.fabs(d[i]) 
        
    # if value is larger than both neighbours , and strictly 
    # larger than either, then it is a local maximum 
    t = [0.0]*len(d) 
    for i in range(len(d)): 
        ll = m[i-1] if i >= 1 else m[i] 
        oo = m[i] 
        rr = m[i+1] if i < len(d)-2 else m[i] 
        if (ll <= oo and oo >= rr) and (ll < oo or oo > rr): 
            # compute magnitude 
            t[i] = math.sqrt(d[i]**2) 
        else: 
            t[i] = 0.0 
            
    return t

In [49]:
def lhipa(d):
    # find max decomposition level
    w = pywt.Wavelet('sym16')
    maxlevel = pywt.dwt_max_level(len(d),filter_len=w.dec_len)
    print(maxlevel)
    
    # set high and low frequency band indeces
    hif, lof = 1, int(maxlevel/2)
    
    # get detail coefficients of pupil diameter signal d
    cD_H = pywt.downcoef('d', d, 'sym16', 'per', level = hif)
    cD_L = pywt.downcoef('d', d, 'sym16', 'per', level = lof)
    
    # normalize by 1/root(2^(j))
    cD_H[:] = [x / math.sqrt(2**hif) for x in cD_H]
    cD_L[:] = [x / math.sqrt(2**lof) for x in cD_L]
    
    # obtain the LH:HF ratio
    cD_LH = cD_L
    for i in range(len(cD_L)):
        cD_LH[i] = cD_L[i] / cD_H[((2**lof)//(2**hif))*i]
        
    # detect modulus maxima, see Duchowski et al.
    cD_LHm = modmax(cD_LH)
    
    # threshold using universal threshold λuniv = σˆ (2 log n)
    # where σˆ is the standard deviation of the noise
    λuniv = np.std(cD_LHm) * math.sqrt(2.0*np.log2(len(cD_LHm))) 
    cD_LHt = pywt.threshold(cD_LHm, λuniv, mode ="less")
    
    # get signal duration (in seconds)
    tt = d[-1].timestamp() - d[0].timestamp()
    
    # compute LHIPA
    ctr = 0
    for i in range(len(cD_LHt)):
        if math.fabs(cD_LHt[i]) > 0: ctr += 1
    LHIPA = float(ctr)/tt
    
    return LHIPA

In [6]:
df = pd.read_csv('/Volumes/Extreme SSD/research data/good system/database tables/gs_local_gaze_pupil_aoi.csv')
df.head()

,imotion_unix_timestamp,user_id,SourceStimuliName,gaze_AOI,Blink detected (binary),ET_pupilAvg
0,1628616621280,P001,A2002,fc_row_5_src_rep,NaN,3.585426
1,1628616621284,P001,A2002,fc_row_5_src_rep,0.0,3.580101
2,1628616621287,P001,A2002,fc_row_5_src_rep,0.0,3.562469
3,1628616621290,P001,A2002,fc_row_5_src_rep,NaN,3.578819
4,1628616621294,P001,A2002,fc_row_5_src_rep,0.0,3.586250


In [68]:
# input format: 2D array, [[pupil1, timestamp1],[pupil2, timestamp2]..]
# input format: 2D array, [[pupil1, pupil2, pupil3,...][timestamp1, timestamp2, timestamp3,...]]

pupil_list = []
ts_list = []
for index, row in df.iterrows():
    if index < 500:
        pupil_list.append(row['ET_pupilAvg'])
        ts_list.append(row['imotion_unix_timestamp'])
    else:
        break
test_list = [pupil_list, ts_list]
len(test_list)
    

2

In [69]:
a = lhipa_new(test_list)
a

4


21.675454012888107

In [62]:
def lhipa_new(d):
    # find max decomposition level
    w = pywt.Wavelet('sym16')
    maxlevel = pywt.dwt_max_level(len(d[0]),filter_len=w.dec_len)
    print(maxlevel)
    
    # set high and low frequency band indeces
    hif, lof = 1, int(maxlevel/2)
    
    # get detail coefficients of pupil diameter signal d
    cD_H = pywt.downcoef('d', d[0], 'sym16', 'per', level = hif)
    cD_L = pywt.downcoef('d', d[0], 'sym16', 'per', level = lof)
    
    # normalize by 1/root(2^(j))
    cD_H[:] = [x / math.sqrt(2**hif) for x in cD_H]
    cD_L[:] = [x / math.sqrt(2**lof) for x in cD_L]
    
    # obtain the LH:HF ratio
    cD_LH = cD_L
    for i in range(len(cD_L)):
        cD_LH[i] = cD_L[i] / cD_H[((2**lof)//(2**hif))*i]
        
    # detect modulus maxima, see Duchowski et al.
    cD_LHm = modmax(cD_LH)
    
    # threshold using universal threshold λuniv = σˆ (2 log n)
    # where σˆ is the standard deviation of the noise
    λuniv = np.std(cD_LHm) * math.sqrt(2.0*np.log2(len(cD_LHm))) 
    cD_LHt = pywt.threshold(cD_LHm, λuniv, mode ="less")
    
    # get signal duration (in seconds)
    tt = (d[1][-1] - d[1][0]) / 1000
    
    # compute LHIPA
    ctr = 0
    for i in range(len(cD_LHt)):
        if math.fabs(cD_LHt[i]) > 0: ctr += 1
    LHIPA = float(ctr)/tt
    
    return LHIPA